In [2]:
%load_ext autoreload
%autoreload 2
import jieba
from jieba import analyse
import transformers as ts
import pandas as pd
import numpy as np
import torch
import json
from sklearn.model_selection import train_test_split
from ark_nlp.model.ner.global_pointer_bert import Dataset as arkData
from ark_nlp.model.ner.global_pointer_bert import Tokenizer,GlobalPointerBertConfig,GlobalPointerBert,get_default_model_optimizer,Task,Predictor
from torch.utils.data import DataLoader,Dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df_train=pd.read_json('./datasets/train.json')
lens=len(df_train)
df_train.head(3)

,text,ner_list,type,doc_id
0,血管内皮生长因子表达水平与宫颈癌患者预后的关系,"[{'mention': '血管内皮生长因子表达水平与宫颈癌患者预后的关系', 'label...",Title,2500
1,目的探讨地尔硫卓在治疗扩张型心肌病中的临床疗效。方法将2014年9月至2017年12月在红河...,"[{'mention': '探讨地尔硫卓在治疗扩张型心肌病中的临床疗效', 'label':...",Abstract,2501
2,地尔硫卓在治疗扩张型心肌病中的临床疗效观察,"[{'mention': '地尔硫卓在治疗扩张型心肌病中的临床疗效观察', 'label':...",Title,2501


In [14]:
#用于W2NER
result=[]
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        text=df_train.loc[i,'text'].replace(' ','')
        #token
        sentence=[char for char in text]
        #实体
        ner=[]
        for n in df_train.loc[i,'ner_list']:
            entity=n['mention'].replace(' ','')
            l=text.index(entity)
            r=l+len(entity)
            index=[j for j in range(l,r)]
            type=n['label']
            ner.append({
                'index':index,
                'type':type,
            })
        #分词
        word=[]
        split_word=jieba.tokenize(text)
        for w in split_word:
            widx=[j for j in range(w[1],w[2])]
            word.append(widx)
        #组装
        result.append({
            'sentence':sentence,
            'ner':ner,
            'word':word,
        })
result_json=json.dumps(result,ensure_ascii = False)
with open('./datasets/w2ner/title_train.json','w',encoding='utf-8')as f:
    f.write(result_json)
    f.close()

In [12]:
bigcat=[]
title_ner=[]
smallcat={
    'C':[],
    'I':[],
    'O':[],
    'P':[],
    'S':[],
}
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        newner=[]
        text=df_train.loc[i,'text'].replace(' ','')
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            newbig={
                'text':ner['mention'],
                'label':l,
            }
            bigcat.append(newbig)
            
            smallcat[l].append({
                'text':ner['mention'],
                'label':ner['label'][2:],
            })
            
            subtext=ner['mention'].replace(' ','')
            sidx=text.find(subtext)
            if sidx == -1:
                continue
            eidx=sidx+len(subtext)
            newner.append({
                'start_idx':sidx,
                'end_idx':eidx,
                'type':ner['label'][0],
                'entity':subtext,
            })
            
        title_ner.append({
            'text':df_train.loc[i,'text'],
            'label':newner,
        })
    else:
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            curtext=ner['mention']
            if len(curtext) > 200:
                todel=len(curtext)-198
                ll=todel//2
                r=todel-ll
                curtext=curtext[ll:-r]
            newbig={
                'text':curtext,
                'label':l,
            }
            bigcat.append(newbig)
            smallcat[l].append({
                'text':curtext,
                'label':ner['label'][2:],
            })
big_json=json.dumps(bigcat,ensure_ascii = False)
with open('./datasets/hierarchical/bigcat_train.json','w',encoding='utf-8') as f:
    f.write(big_json)
    f.close()
    
title_json=json.dumps(title_ner,ensure_ascii = False)
with open('./datasets/hierarchical/titlener_train.json','w',encoding='utf-8') as f:
    f.write(title_json)
    f.close()
    
for key in smallcat.keys():
    small_json=json.dumps(smallcat[key],ensure_ascii = False)
    with open('./datasets/hierarchical/'+key+'cat_train.json','w',encoding='utf-8') as f:
        f.write(small_json)
        f.close()

In [ ]:
ttt="采用抽签法将安阳市中医院心病一科2014年3月~2015年4月113例不稳定型心绞痛患者进行分组,对照组56例给予疏血通注射液6mL/d治疗,观察组57例给予疏血通注射液10mL/d治疗,观察两组临床疗效及不良反应"
analyse.extract_tags(ttt,topK=30)